# Practico Exploracion y curacion

-----

## Consignas

### 1. Curación de datos anómalos

Existe una cantidad de datos anómalos en el dataset y es necesario curarlos. Elegir y justificar una estrategia de imputación para los datos nulos.

### 2. Eliminación de outliers: 

En base en el estudio de outliers hecho en el practico 1, definan cotas inferiores y superiores para las variables que presenten outliers y filtren las filas que no cumplan con esos limites.

### 3. Balanceo de etiquetas:

La etiqueta positiva esta desbalanceada con respecto a la negativa. Esto se vio en el análisis de distribución sobre las distintas variables con respecto a la etiqueta. Existen distintos métodos para balanceo de etiquetas (ver notebooks over_sampling y under_sampling) así que vamos a probar algunos para ayudar a nuestros modelos del practico que viene.

- Usar al menos tres métodos de over-sampling para balancear el target. Uno de ellos tiene que ser RandomOverSampler
- Usar al menos tres métodos de under-sampling para balancear el target. Uno de ellos tiene que ser RandomUnderSampler

### 4. Escalado

Todas las métricas provenientes de la teoría de la información con excepción de ZE están escaladas entre 0 y 1. Es decir que tienen un mínimo en 0 y máximo en 1. Es necesario que todas nuestras features tengan la misma escala para que los modelos de ML funcionen mejor. Escalar LZE y las métricas de estadística clásica (media, varianza, skew) entre 0 y 1. (hint: MinMaxScaler)

### 5. PCA

Realizar PCA sobre el dataset original y acoplar las columnas generadas mas significativas al dataset. 

### Referencias

PCA: https://towardsdatascience.com/principal-component-analysis-3c39fbf5cb9d

Etiquetas no balanceadas: https://towardsdatascience.com/practical-tips-for-class-imbalance-in-binary-classification-6ee29bcdb8a7#:~:text=3.%20Resampling%20to,imbalanced%2Dlearn%20library

imbalanced-learn: https://github.com/scikit-learn-contrib/imbalanced-learn